## format and split data

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [2]:
train = pd.read_csv('train_small.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.dropna(subset=['type'], inplace=True)

In [4]:
test = pd.read_csv('labeled_output.csv')

In [5]:
test.dropna(subset=['text'], inplace=True)

In [6]:
test.drop(columns=['text_type'], inplace=True)

In [7]:
X = train['text']
y = train['type']

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=32, test_size=0.2)

In [9]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.fit_transform(y_test)

In [10]:
import keras
from sklearn.preprocessing import LabelEncoder

num_classes=5
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vec = TfidfVectorizer(analyzer='word',
                         min_df=1,
                         strip_accents='unicode',
                         token_pattern=r'\w{1,}',
                         ngram_range=(2,3),
                         )
x_train_vec = tf_vec.fit_transform(x_train)
x_test_vec = tf_vec.transform(x_test)
print (x_train_vec.shape)
print (x_test_vec.shape)

(207, 2163)
(52, 2163)


## neural net

In [12]:
from keras import models
from keras import layers
from keras.layers import Dropout
from keras import optimizers
from keras.optimizers import SGD

In [17]:
def create_model():
    inputs = layers.Input(shape=(2163,))
    hidden_1 = layers.Dense(units=32, activation='relu')(inputs)
    dropout1 = layers.Dropout(0.5)(hidden_1)
    hidden_2 = layers.Dense(units=32, activation='relu')(hidden_1)
    dropout_2 = layers.Dropout(0.5)(hidden_2)
    hidden_3 = layers.Dense(units=32, activation='relu')(hidden_2)
    dropout_3 = layers.Dropout(0.5)(hidden_3)
    hidden_4 = layers.Dense(units=32, activation='relu')(hidden_3)
    dropout_4 = layers.Dropout(0.5)(hidden_3)
    hidden_5 = layers.Dense(units=32, activation='relu')(hidden_4)
    dropout_5 = layers.Dropout(0.5)(hidden_4)
    outputs = layers.Dense(5, activation='softmax')(hidden_5)

    model= models.Model(inputs=inputs, outputs=outputs)
    model.summary()
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')
    return model

## parameter training

In [18]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model, verbose=0)

In [19]:
from sklearn.model_selection import GridSearchCV

batch_size= [5,8,10,15,25,35,50]
epochs=[2,4,6,8,10,12,40]

param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [20]:
grid_pred = grid.fit(x_train_vec, y_train);

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2163)              0         
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
_________________________________________________________________
dense_1 (Dense)              (None, 32)                69248     
input_1 (InputLayer)         (None, 2163)              0         
_________________________________________________________________
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
Layer (type)                 Output Shape              Param #   
dense_1 (D

_________________________________________________________________
_________________________________________________________________
dense_1 (Dense)              (None, 32)                69248     
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
dense_5 (D

dense_11 (Dense)             (None, 32)                1056      
dense_8 (Dense)              (None, 32)                1056      
_________________________________________________________________
_________________________________________________________________
dense_12 (Dense)             (None, 5)                 165       
dense_9 (Dense)              (None, 32)                1056      
_________________________________________________________________
Total params: 73,637
dense_10 (Dense)             (None, 32)                1056      
Trainable params: 73,637
_________________________________________________________________
Non-trainable params: 0
_________________________________________________________________
dense_11 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_12 (Dense)             (None, 5)                 165       
Total params: 73,637
Trainable params: 73,637
Non-trainable params: 0
__

Trainable params: 73,637
Non-trainable params: 0
dense_17 (Dense)             (None, 32)                1056      
_________________________________________________________________
_________________________________________________________________
dense_18 (Dense)             (None, 5)                 165       
Total params: 73,637
Trainable params: 73,637
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 2163)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)                69248     
_________________________________________________________________
dense_14 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_15 (Dense)       

Trainable params: 73,637
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
dense_19 (Dense)             (None, 32)                69248     
_________________________________________________________________
dense_20 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_21 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_22 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_23 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_24 (Dense)             (None, 5)                 165       
Total params: 73,637
Trainable params: 73,637
Non-trainable params: 0
_______________________

...

_________________________________________________________________
dense_88 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_89 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_90 (Dense)             (None, 5)                 165       
Total params: 73,637
Trainable params: 73,637
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 2163)              0         
_________________________________________________________________
dense_85 (Dense)             (None, 32)                69248     
_________________________________________________________________
dense_86 (Dense)             (None, 32)                1056      
______

Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 2163)              0         
_________________________________________________________________
dense_91 (Dense)             (None, 32)                69248     
_________________________________________________________________
dense_92 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_93 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_94 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_95 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_96 (Dense)             (None, 5)                 165       
Total params: 73,637
Trainable params: 73,637
Non-trainable params: 0
______

dense_100 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_101 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_102 (Dense)            (None, 5)                 165       
Total params: 73,637
Trainable params: 73,637
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 2163)              0         
_________________________________________________________________
dense_97 (Dense)             (None, 32)                69248     
_________________________________________________________________
dense_98 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_

input_18 (InputLayer)        (None, 2163)              0         
_________________________________________________________________
dense_103 (Dense)            (None, 32)                69248     
_________________________________________________________________
dense_104 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_105 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_106 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_107 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_108 (Dense)            (None, 5)                 165       
Total params: 73,637
Trainable params: 73,637
Non-trainable params: 0
_________________________________________________________________
______

_________________________________________________________________
dense_107 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_108 (Dense)            (None, 5)                 165       
Total params: 73,637
Trainable params: 73,637
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 2163)              0         
_________________________________________________________________
dense_109 (Dense)            (None, 32)                69248     
_________________________________________________________________
dense_110 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_111 (Dense)            (None, 32)                1056      
______

In [21]:
print(grid_pred.best_score_)
print(grid_pred.best_params_)

0.8164251144381537
{'batch_size': 10, 'epochs': 40}


In [ ]:
#model.fit(x=x_train_vec, y=y_train, batch_size=?, epochs=3?)
#test_loss, test_acc = model.evaluate(x_test_vec, y_test)
#test_acc